# Loop 7 Analysis: Rotation Optimization and bbox3

The evaluator identified two key techniques NOT yet tried:
1. **Rotation optimization (fix_direction)** - Optimize the rotation angle of the entire configuration
2. **bbox3 optimizer** - A fundamentally different optimization approach

Let's analyze these approaches.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
from scipy.optimize import minimize_scalar
from shapely.geometry import Polygon
from shapely import affinity

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

print('Functions defined')

Functions defined


In [2]:
# Load the current best baseline
df = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
print(f'Loaded {len(df)} rows')
print(df.head())

Loaded 20100 rows
      id                       x                       y  \
0  001_0    s-48.196086194214246     s58.770984615214225   
1  002_0   s0.154097069621355887  s-0.038540742694794648   
2  002_1  s-0.154097069621372845  s-0.561459257305224058   
3  003_0      s1.123655816140301      s0.781101815992563   
4  003_1       s1.23405569584216      s1.275999500663759   

                       deg  
0                    s45.0  
1  s203.629377730656841550  
2   s23.629377730656791812  
3        s111.125132292893  
4         s66.370622269343  


In [3]:
# Implement rotation optimization (fix_direction)
def calculate_bbox_side_at_angle(angle_deg, points):
    """Calculate bounding box side length at a given rotation angle."""
    angle_rad = np.radians(angle_deg)
    c, s = np.cos(angle_rad), np.sin(angle_rad)
    rot_matrix = np.array([[c, s], [-s, c]])
    rotated_points = points @ rot_matrix.T
    min_xy = np.min(rotated_points, axis=0)
    max_xy = np.max(rotated_points, axis=0)
    return max(max_xy[0] - min_xy[0], max_xy[1] - min_xy[1])

def optimize_rotation_for_n(df, n):
    """Optimize rotation angle for configuration N."""
    prefix = f'{n:03d}_'
    trees = df[df['id'].str.startswith(prefix)]
    
    if len(trees) != n:
        return None, None, None
    
    # Get all polygon vertices
    all_points = []
    for _, row in trees.iterrows():
        x = parse_value(row['x'])
        y = parse_value(row['y'])
        deg = parse_value(row['deg'])
        poly = create_tree_polygon(x, y, deg)
        all_points.extend(list(poly.exterior.coords))
    
    points = np.array(all_points)
    
    # Get convex hull for efficiency
    try:
        hull_points = points[ConvexHull(points).vertices]
    except:
        hull_points = points
    
    # Calculate initial side
    initial_side = calculate_bbox_side_at_angle(0, hull_points)
    
    # Optimize rotation
    result = minimize_scalar(
        lambda a: calculate_bbox_side_at_angle(a, hull_points),
        bounds=(0, 90),
        method='bounded'
    )
    
    best_angle = result.x
    best_side = result.fun
    improvement = initial_side - best_side
    
    return initial_side, best_side, best_angle, improvement

print('Rotation optimization function defined')

Rotation optimization function defined


In [4]:
# Test rotation optimization on a few N values
test_ns = [1, 10, 50, 100, 150, 200]
print('Testing rotation optimization on selected N values:\n')

for n in test_ns:
    result = optimize_rotation_for_n(df, n)
    if result[0] is not None:
        initial, best, angle, improvement = result
        print(f'N={n:3d}: Initial={initial:.6f}, Best={best:.6f}, Angle={angle:.4f}°, Improvement={improvement:.9f}')

Testing rotation optimization on selected N values:

N=  1: Initial=0.813173, Best=0.813173, Angle=90.0000°, Improvement=-0.000000051
N= 10: Initial=1.940696, Best=1.940696, Angle=0.0000°, Improvement=-0.000000105
N= 50: Initial=4.247076, Best=4.247077, Angle=90.0000°, Improvement=-0.000000461
N=100: Initial=5.878188, Best=5.878188, Angle=0.0000°, Improvement=-0.000000256
N=150: Initial=7.110529, Best=7.110529, Angle=0.0000°, Improvement=0.000000011
N=200: Initial=8.216619, Best=8.216620, Angle=0.0000°, Improvement=-0.000000440


In [5]:
# Apply rotation optimization to ALL N values and calculate total improvement
print('Applying rotation optimization to all N values...')

total_improvement = 0
improvements = []

for n in range(1, 201):
    result = optimize_rotation_for_n(df, n)
    if result[0] is not None:
        initial, best, angle, improvement = result
        # Score contribution: side^2 / n
        initial_score = initial**2 / n
        best_score = best**2 / n
        score_improvement = initial_score - best_score
        total_improvement += score_improvement
        improvements.append({
            'n': n,
            'initial_side': initial,
            'best_side': best,
            'angle': angle,
            'side_improvement': improvement,
            'score_improvement': score_improvement
        })

print(f'\nTotal score improvement from rotation: {total_improvement:.9f}')
print(f'Current score: 70.659959')
print(f'New score after rotation: {70.659959 - total_improvement:.6f}')

Applying rotation optimization to all N values...



Total score improvement from rotation: -0.000006038
Current score: 70.659959
New score after rotation: 70.659965


In [6]:
# Show top 10 N values with most improvement from rotation
import pandas as pd
imp_df = pd.DataFrame(improvements)
imp_df = imp_df.sort_values('score_improvement', ascending=False)
print('Top 10 N values with most improvement from rotation:')
print(imp_df.head(10).to_string(index=False))

Top 10 N values with most improvement from rotation:
  n  initial_side  best_side     angle  side_improvement  score_improvement
128      6.633064   6.633060  0.000443      3.850915e-06       3.991150e-07
 88      5.555095   5.555094  0.000122      9.701032e-07       1.224776e-07
 87      5.547459   5.547458 89.999934      8.942081e-07       1.140364e-07
178      7.715610   7.715609  0.000097      8.168857e-07       7.081765e-08
176      7.715196   7.715195  0.000034      2.951167e-07       2.587367e-08
132      6.638062   6.638062  0.000015      1.277608e-07       1.284976e-08
130      6.636508   6.636508  0.000011      9.676777e-08       9.880002e-09
177      7.715550   7.715550  0.000007      6.173760e-08       5.382367e-09
 71      5.000850   5.000850  0.000004      2.344306e-08       3.302401e-09
150      7.110529   7.110529  0.000004      1.074813e-08       1.018998e-09


In [7]:
# Summary of rotation optimization potential
print('='*60)
print('ROTATION OPTIMIZATION SUMMARY')
print('='*60)
print(f'Total score improvement: {total_improvement:.9f}')
print(f'N values with improvement > 1e-6: {len(imp_df[imp_df.score_improvement > 1e-6])}')
print(f'N values with improvement > 1e-9: {len(imp_df[imp_df.score_improvement > 1e-9])}')
print(f'Average improvement per N: {total_improvement/200:.12f}')
print('='*60)
print(f'\nConclusion: Rotation optimization provides {total_improvement:.9f} improvement.')
print(f'This is {"significant" if total_improvement > 0.001 else "negligible"} compared to the 1.74 gap to target.')

ROTATION OPTIMIZATION SUMMARY
Total score improvement: -0.000006038
N values with improvement > 1e-6: 0
N values with improvement > 1e-9: 10
Average improvement per N: -0.000000030191

Conclusion: Rotation optimization provides -0.000006038 improvement.
This is negligible compared to the 1.74 gap to target.


In [ ]:
# Let's analyze the bbox3 C++ code to understand its approach
# The key insight is that bbox3 uses fundamentally different techniques:
# 1. Complex number vector coordination
# 2. Fluid dynamics
# 3. Hinge pivot
# 4. Density gradient flow
# 5. Global boundary tension

# Let's extract the bbox3 code and compile it
import os

bbox3_code = '''
// BBOX3 - Global Dynamics Edition
// Features: Complex Number Vector Coordination, Fluid Dynamics, Hinge Pivot, 
// Density Gradient Flow, and NEW Global Boundary Tension.

#include <iostream>
#include <fstream>
#include <sstream>
#include <cmath>
#include <algorithm>
#include <string>
#include <vector>
#include <map>
#include <set>
#include <tuple>
#include <iomanip>
#include <chrono>
#include <random>
#include <numeric>
#include <omp.h>
#include <complex> 

using namespace std;
using namespace chrono;

constexpr int MAX_N = 200;
constexpr int NV = 15;
constexpr double PI = 3.14159265358979323846;
constexpr double EPSILON = 1e-16;
constexpr double NEIGHBOR_RADIUS = 0.5;      
constexpr double PIVOT_ANGLE_MAX = 10.0;     
constexpr double GLOBAL_TENSION_STRENGTH = 0.05; 

// Base tree geometry 
const double TX[NV] = {0,0.125,0.0625,0.2,0.1,0.35,0.075,0.075,-0.075,-0.075,-0.35,-0.1,-0.2,-0.0625,-0.125};
const double TY[NV] = {0.8,0.5,0.5,0.25,0.25,0,0,-0.2,-0.2,0,0,0.25,0.25,0.5,0.5};

thread_local mt19937_64 rng(44); 
thread_local uniform_real_distribution<double> U(0, 1);

inline double rf() { return U(rng); }
inline int ri(int n) { return rng() % n; }

using Complex = std::complex<double>;

struct Poly {
    Complex p[NV]; 
    double x0, y0, x1, y1;
    void bbox() {
        x0 = x1 = p[0].real(); y0 = y1 = p[0].imag();
        for (int i = 1; i < NV; i++) {
            x0 = min(x0, p[i].real()); x1 = max(x1, p[i].real());
            y0 = min(y0, p[i].imag()); y1 = max(y1, p[i].imag());
        }
    }
};

Poly getPoly(Complex c_center, double deg) {
    Poly q;
    double r = deg * PI / 180;
    Complex c_rot = polar(1.0, r); 

    for (int i = 0; i < NV; i++) {
        Complex base_pt(TX[i], TY[i]);
        Complex rotated_pt = base_pt * c_rot; 
        q.p[i] = rotated_pt + c_center;
    }
    q.bbox();
    return q;
}

bool pip(double px, double py, const Poly& q) {
    bool in = false;
    int j = NV - 1;
    for (int i = 0; i < NV; i++) {
        double qi_x = q.p[i].real(), qi_y = q.p[i].imag();
        double qj_x = q.p[j].real(), qj_y = q.p[j].imag();
        if ((qi_y > py) != (qj_y > py) &&
            px < (qj_x - qi_x) * (py - qi_y) / (qj_y - qi_y) + qi_x)
            in = !in;
        j = i;
    }
    return in;
}

bool segInt(Complex a, Complex b, Complex c, Complex d) {
    auto ccw = [](Complex p, Complex q, Complex r) { 
        return (r.imag() - p.imag()) * (q.real() - p.real()) > (q.imag() - p.imag()) * (r.real() - p.real()); 
    };
    return ccw(a, c, d) != ccw(b, c, d) && ccw(a, b, c) != ccw(a, b, d);
}

bool overlap(const Poly& a, const Poly& b) {
    if (a.x1 < b.x0 || b.x1 < a.x0 || a.y1 < b.y0 || b.y1 < a.y0) return false;
    for (int i = 0; i < NV; i++) {
        if (pip(a.p[i].real(), a.p[i].imag(), b)) return true;
        if (pip(b.p[i].real(), b.p[i].imag(), a)) return true;
    }
    for (int i = 0; i < NV; i++)
        for (int j = 0; j < NV; j++)
            if (segInt(a.p[i], a.p[(i + 1) % NV], b.p[j], b.p[(j + 1) % NV])) return true;
    return false;
}

struct Cfg {
    int n;
    Complex c[MAX_N]; 
    double a[MAX_N];  
    Poly pl[MAX_N];

    void upd(int i) { pl[i] = getPoly(c[i], a[i]); }
    void updAll() { for (int i = 0; i < n; i++) upd(i); }

    bool hasOvl(int i) const {
        for (int j = 0; j < n; j++) { 
            if (i != j && overlap(pl[i], pl[j])) return true;
        }
        return false;
    }

    bool anyOvl() const {
        for (int i = 0; i < n; i++)
            for (int j = i + 1; j < n; j++)
                if (overlap(pl[i], pl[j])) return true;
        return false;
    }

    double side() const {
        if (!n) return 0;
        double x0 = pl[0].x0, x1 = pl[0].x1, y0 = pl[0].y0, y1 = pl[0].y1;
        for (int i = 1; i < n; i++) {
            x0 = min(x0, pl[i].x0); x1 = max(x1, pl[i].x1);
            y0 = min(y0, pl[i].y0); y1 = max(y1, pl[i].y1);
        }
        return max(x1 - x0, y1 - y0);
    }

    double score() const { double s = side(); return s * s / n; }

    Complex centroid() const {
        Complex sum = 0.0;
        for (int i = 0; i < n; i++) { sum += c[i]; }
        return sum / (double)n;
    }
};

// Simple SA optimization
double run_sa(Cfg& cur, int iterations, double scale) {
    double best_s = cur.side();
    
    for (int it = 0; it < iterations; it++) {
        int i = ri(cur.n);
        Complex oc = cur.c[i];
        double oa = cur.a[i];
        
        double r = rf();
        if (r < 0.5) {
            cur.c[i] += Complex((rf()-0.5) * scale, (rf()-0.5) * scale);
        } else {
            cur.a[i] += (rf()-0.5) * scale * 45.0;
        }
        
        cur.upd(i);
        
        if (!cur.hasOvl(i) && cur.side() <= best_s + 1e-15) {
            if (cur.side() < best_s - 1e-15) best_s = cur.side();
        } else {
            cur.c[i] = oc;
            cur.a[i] = oa;
            cur.upd(i);
        }
    }
    return best_s;
}

// IO
map<int, Cfg> loadCSV(string fn) {
    map<int, Cfg> res; ifstream f(fn); string ln, h; if(!f) return res;
    getline(f, h);
    while (getline(f, ln)) {
        stringstream ss(ln); string id, sx, sy, sa;
        if(!getline(ss, id, ',')) continue;
        getline(ss, sx, ','); getline(ss, sy, ','); getline(ss, sa, ',');
        int n = stoi(id.substr(0, 3)), idx = stoi(id.substr(4));
        auto p = [](string s) { 
            size_t st = s.find_first_of("0123456789.-"); 
            return (st == string::npos) ? 0.0 : stod(s.substr(st)); 
        };
        res[n].n = n; 
        res[n].c[idx] = Complex(p(sx), p(sy));
        res[n].a[idx] = p(sa);
    }
    for (auto& pair : res) pair.second.updAll();
    return res;
}

void saveCSV(string fn, map<int, Cfg>& res) {
    ofstream f(fn); f << "id,x,y,deg" << endl;
    for (int n = 1; n <= 200; n++) {
        if (!res.count(n)) continue;
        for (int i = 0; i < n; i++)
            f << setfill('0') << setw(3) << n << "_" << i << ",s" << fixed << setprecision(18) 
              << res[n].c[i].real() << ",s" << res[n].c[i].imag() << ",s" << res[n].a[i] << "\\n";
    }
}

int main(int argc, char* argv[]) {
    string input_file = "submission.csv";
    int n_iter = 1000;
    int r_value = 30;
    
    for (int i = 1; i < argc; i++) {
        string arg = argv[i];
        if (arg == "-n" && i+1 < argc) n_iter = stoi(argv[++i]);
        else if (arg == "-r" && i+1 < argc) r_value = stoi(argv[++i]);
        else input_file = arg;
    }
    
    auto res = loadCSV(input_file);
    if(res.empty()) {
        cerr << "Failed to load " << input_file << endl;
        return 1;
    }
    
    double init_score = 0;
    for (auto& [n, cfg] : res) init_score += cfg.score();
    cout << "Initial score: " << fixed << setprecision(6) << init_score << endl;
    
    vector<int> keys; for(auto const& [n, g] : res) keys.push_back(n);
    sort(keys.rbegin(), keys.rend());

    vector<double> scales = {1e-2, 1e-3, 1e-4, 1e-5, 1e-6};
    for (double sc : scales) {
        cout << "Scale: " << sc << endl;
        #pragma omp parallel for schedule(dynamic, 1)
        for (int i = 0; i < (int)keys.size(); i++) {
            int n = keys[i];
            run_sa(res[n], n_iter * r_value, sc);
        }
    }
    
    double final_score = 0;
    for (auto& [n, cfg] : res) final_score += cfg.score();
    cout << "Final score: " << fixed << setprecision(6) << final_score << endl;
    cout << "Improvement: " << fixed << setprecision(9) << (init_score - final_score) << endl;
    
    saveCSV("submission.csv", res);
    return 0;
}
'''

print("bbox3 code analyzed. Key differences from Eazy optimizer:")\nprint("1. Uses complex numbers for position representation")\nprint("2. Simpler SA approach but with multi-scale optimization")\nprint("3. Parameters: -n (iterations), -r (multiplier)")\nprint("\\nThe bbox3-runner workflow:")\nprint("1. Run bbox3 with various n,r combinations")\nprint("2. Apply fix_direction (rotation optimization)")\nprint("3. Repair overlaps with donor solutions")\nprint("4. Keep best result")